## Load Packages and Classes

In [1]:
import sys
sys.path.append('/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose')

from data_scripts.data_type_classes.RADARDataCapture import FMCWRADARDataCapture
from data_scripts.data_type_classes.MOCAPDataCapture import MOCAPDataCapture
from data_scripts.data_type_classes.FPDataCapture import FPDataCapture
from data_scripts.data_type_classes.tsv_processor_knee_angles import TSVProcessorKnee
import h5py
import os
import numpy as np
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Load Key Frames and Times Dataframe

In [7]:
df_key_capture_times_frames = pd.read_csv('/Volumes/FourTBLaCie/RADARTreePose_Data/metadata/FULL_MNTRR_MNTRL_key_times_frames.csv')
df_key_capture_times_frames

,Unnamed: 0,RADAR_capture,MOCAP_Start_Time,MOCAP_End_Time,RADAR_Start_Frame,RADAR_End_Frame,Seconds_per_Frame,tx,is_final_tx,t_foot_up,t_stable,t_break,t_foot_down,frame_foot_up,frame_stable,frame_break,frame_end
0,0,01_MNTRL_RR_V1,7.33,30.57,194,831,0.03645,1,False,10.22000,11.19,14.28,15.05000,273,300,384.0,405
1,1,01_MNTRL_RR_V1,7.33,30.57,194,831,0.03645,2,False,18.25000,19.05,22.18,22.83000,493,515,601.0,619
2,2,01_MNTRL_RR_V1,7.33,30.57,194,831,0.03645,3,True,26.24000,27.03,NaN,30.57000,712,734,NaN,831
3,3,01_MNTRL_RR_V2,7.37,30.40,191,822,0.03646,1,False,10.48000,11.05,14.27,15.05000,276,292,380.0,401
4,4,01_MNTRL_RR_V2,7.37,30.40,191,822,0.03646,2,False,18.41000,19.07,22.17,22.86000,493,511,596.0,615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,284,31_MNTRR_RR_V2,7.63,30.20,190,783,0.03811,2,False,17.94000,18.57,22.24,22.95500,460,477,573.0,592
285,285,31_MNTRR_RR_V2,7.63,30.20,190,783,0.03811,3,True,26.02833,27.23,NaN,30.20000,672,704,NaN,782
286,286,31_MNTRR_RR_V3,7.60,30.70,157,789,0.03660,1,False,10.25750,11.55,14.16,14.58083,229,265,336.0,347
287,287,31_MNTRR_RR_V3,7.60,30.70,157,789,0.03660,2,False,18.22250,18.76,22.40,22.87250,447,462,561.0,574


### Generate subselected RDMs From Radar Data

In [9]:
output_folder_path = "/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_v1"
output_folder_path_one_channel = "/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_by_channel_v1"

problem_list = []

for index, row in df_key_capture_times_frames.iterrows():
    
    capture_info = df_key_capture_times_frames[df_key_capture_times_frames['RADAR_capture']==row['RADAR_capture']]    
    RADAR_start_frame = int(capture_info['RADAR_Start_Frame'].iloc[0])
    RADAR_end_frame = int(capture_info['RADAR_End_Frame'].iloc[0])
    
    participant = row['RADAR_capture'][:2]
    
    radar_file_path = f"/Volumes/FourTBLaCie/RADARTreePose_Data/raw/radar/{participant}/{row['RADAR_capture']}.h5"
    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        print(e)
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue
    
    # Load and save the data
    dataCubes = RADAR_object.load_and_save()

    # Process the data with Range-Doppler processing
    processed_data = RADAR_object.range_doppler_processing(dataCubes)
    
    print(f"Range doppler processed shape: {processed_data.shape}")
    
    sub_selected_processed_data = RADAR_object.sub_select_RADAR_DATA(processed_data)
    
    print(f"Sub-Selected Range doppler processed shape: {sub_selected_processed_data.shape}")
    
     # Convert the list of processed frames into a 3D numpy array
    processed_frames_array = np.array(sub_selected_processed_data)
    
    print(f"Npy Sub-Selected Range doppler processed shape: {processed_frames_array.shape}")
    
    # Ensure the output folder exists
    specific_output_folder_path = os.path.join(output_folder_path, participant)
    if not os.path.exists(specific_output_folder_path):
        os.makedirs(specific_output_folder_path)
    
    # Define the output file name for the processed data of the current transition
    output_filename = os.path.join(
        specific_output_folder_path,
        f"{row['RADAR_capture']}.npy"
    )
    # Save the 3D array to a binary file in NumPy .npy format
    np.save(output_filename, processed_frames_array)
    
    print(f"{row['RADAR_capture']} processed and saved")
    
    for i, channel in enumerate(processed_frames_array):
        print(channel.shape)
            # Ensure the output folder exists
        specific_output_folder_path_one_channel = os.path.join(output_folder_path_one_channel, participant)
        if not os.path.exists(specific_output_folder_path_one_channel):
            os.makedirs(specific_output_folder_path_one_channel)
            
            # Define the output file name for the processed data of the current transition
        output_filename_channel = os.path.join(
            specific_output_folder_path_one_channel,
            f"{row['RADAR_capture']}_channel{i+1}.npy"
        )
        # Save the 3D array to a binary file in NumPy .npy format
        np.save(output_filename_channel, channel)
        print(output_filename_channel)
        

print(f"Problem with {problem_list}")
        
    

Range doppler processed shape: (4, 1000, 128, 128)
Sub-Selected Range doppler processed shape: (4, 1000, 23, 13)
Npy Sub-Selected Range doppler processed shape: (4, 1000, 23, 13)
01_MNTRL_RR_V1 processed and saved
(1000, 23, 13)
/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_by_channel_v1/01/01_MNTRL_RR_V1_channel1.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_by_channel_v1/01/01_MNTRL_RR_V1_channel2.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_by_channel_v1/01/01_MNTRL_RR_V1_channel3.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/RDMs_npy_by_channel_v1/01/01_MNTRL_RR_V1_channel4.npy
Range doppler processed shape: (4, 1000, 128, 128)
Sub-Selected Range doppler processed shape: (4, 1000, 23, 13)
Npy Sub-Selected Range doppler processed shape: (4, 1000, 23, 13)
01_MNTRL_RR_V1 processed and saved
(1000, 23, 13)
/Volumes/FourTBLaCie/RADARTreePose_Data/processed/radar/R